# Python Script for NBA Players data
### Alexander Williamson and Brenton Wilder
### Fall Semester 2020

In [1]:
# Load the dataset directly from repository
import pandas as pd

url = 'https://github.com/bwilder95/NBAplayers/blob/main/Players.csv?raw=true'
df = pd.read_csv(url,sep=',')
print(df.head())

   Unnamed: 0           Player  height  weight  \
0           0  Curly Armstrong   180.0    77.0   
1           1     Cliff Barker   188.0    83.0   
2           2    Leo Barnhorst   193.0    86.0   
3           3       Ed Bartels   196.0    88.0   
4           4      Ralph Beard   178.0    79.0   

                           collage    born   birth_city birth_state  
0               Indiana University  1918.0          NaN         NaN  
1           University of Kentucky  1921.0     Yorktown     Indiana  
2         University of Notre Dame  1924.0          NaN         NaN  
3  North Carolina State University  1925.0          NaN         NaN  
4           University of Kentucky  1927.0  Hardinsburg    Kentucky  


In [2]:
# Confirm shape of dataset is correct (3922 rows and 8 columns)
"""
Column 1 (index 0) is unamed
Column 2 (index 1) is Player
Column 3 (index 2) is collage
Column 4 (index 3) is height
Column 5 (index 4) is weight
Column 6 (index 5) is born
Column 7 (index 6) is birth_city
Column 8 (index 7) is birth_state
"""
df.shape

(3922, 8)

In [14]:
# Task 1: Modify pandas column header to fix collage to college. Also, replace blank column with ID.
df.rename(columns={'Unnamed: 0': 'ID', 'collage': 'college'}, inplace=True)